## Binary classification

In [17]:
import numpy as np
import scipy.sparse as sp
import matplotlib
import math
matplotlib.use("svg")
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [40]:
# init
filename="features/blues/blues.00000.csv"
# filename="feature_txt/blues/blues.00000.txt"
X = np.loadtxt(filename,delimiter=',')
print X.shape
y=np.ones(X.shape[0])
print y.shape

# filenames=["classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]
filenames=["classical"]
for filename in filenames:
#     X0=np.loadtxt("feature_txt/"+filename+"/"+filename+".00000.txt",delimiter=',')
    X0=np.loadtxt("features/"+filename+"/"+filename+".00000.csv",delimiter=',')
    X=np.vstack((X,X0))
    y=np.concatenate((y,np.zeros(X0.shape[0])))
print X.shape
print y.shape

(3006, 34)
(3006,)
(6012, 34)
(6012,)


In [41]:
class SVM:
    def __init__(self, X, y, reg):
        """ Initialize the SVM attributes and initialize the weights vector to the zero vector. 
            Attributes: 
                X (array_like) : training data intputs
                y (vector) : 1D numpy array of training data outputs
                reg (float) : regularizer parameter
                theta : 1D numpy array of weights
        """
        self.X = sp.csr_matrix(X)
        self.y = sp.csr_matrix(y)
        self.reg = reg
        self.theta = sp.csr_matrix(np.zeros(X.shape[1]))
        
    
    def objective(self, X, y):
        """ Calculate the objective value of the SVM. When given the training data (self.X, self.y), this is the 
            actual objective being optimized. 
            Args:
                X (array_like) : array of examples, where each row is an example
                y (array_like) : array of outputs for the training examples
            Output:
                (float) : objective value of the SVM when calculated on X,y
        """
        y_coo=sp.csr_matrix(y)
        X_coo=sp.csr_matrix(X)
        y_my_coo=X_coo.dot(self.theta.T)
        ones=sp.csr_matrix(np.ones(y.shape[1]))
        tmp=sp.csr_matrix(ones-y_coo.multiply(y_my_coo.T))
        positive=(tmp>0)+0
        return (tmp.multiply(positive)).sum()+0.5*self.reg*np.linalg.norm(self.theta)**2
        
        
    def gradient(self):
        """ Calculate the gradient of the objective value on the training examples. 
            Output:
                (vector) : 1D numpy array containing the gradient
        """
        y_my_coo=(self.X).dot(self.theta.T)
        product_y=(self.y).multiply(y_my_coo.T)
        ones=sp.csr_matrix(np.ones(product_y.shape))
        mask=ones-((product_y>1)+0)  # 0/1
        coe_X_coo=(self.y).multiply(mask)*(-1)
        return sp.csr_matrix(coe_X_coo.dot(self.X)+self.reg*self.theta).toarray()[0]

        
    def train(self, niters=100, learning_rate=1, verbose=False):
        """ Train the support vector machine with the given parameters. 
            Args: 
                niters (int) : the number of iterations of gradient descent to run
                learning_rate (float) : the learning rate (or step size) to use when training
                verbose (bool) : an optional parameter that you can use to print useful information (like objective value)
        """
        for i in range(niters):
            gradient=self.gradient()
            self.theta=self.theta-learning_rate*gradient
            if verbose:
                print 'obj=',self.objective(self.X,self.y)
            
    
    def predict(self, X):
        """ Predict the class of each label in X. 
            Args: 
                X (array_like) : array of examples, where each row is an example
            Output:
                (vector) : 1D numpy array containing predicted labels
        """
        X_coo=sp.csr_matrix(X)
        y_my_coo=X_coo.dot(self.theta.T).T
        return sp.csr_matrix(((y_my_coo>=0)+0)*2-1).toarray()[0]

In [42]:
m=X.shape[0]
split=int(math.floor(m*0.9))
P=np.random.permutation(m)
X_tr=X[P[0:split]]
X_te=X[P[split:m]]
y_tr=y[P[0:split]]
y_te=y[P[split:m]]
reg=1e-4
svm=SVM(X_tr, y_tr, reg)
svm.train()
y_p=svm.predict(X_te)
accuracy=(y_p==y_te).sum()/float(len(y_te))
print 'accuracy =',accuracy

accuracy = 0.509966777409


## Multi-class

In [21]:
# init
# n=13
n=34
X=np.ones((0,n))
y=np.asarray([])
filenames=["blues","classical","country","disco","hiphop","jazz","metal","pop","reggae","rock"]
for i,filename in enumerate(filenames):
#     X0=np.loadtxt("feature_txt/"+filename+"/"+filename+".00000.txt",delimiter=',')
    X0=np.loadtxt("features/"+filename+"/"+filename+".00000.csv",delimiter=',')
    X=np.vstack((X,X0))
    y=np.concatenate((y,np.repeat(i,X0.shape[0])))
print X.shape
print y.shape

(30075, 34)
(30075,)


In [22]:
def softmax_loss(X, Theta, y, return_grad=False):
    """
    Compute softmax loss and its gradient.

    Args:
        X : 2D numpy array, size m x n: array of all input features for all examples
        Theta: 2D numpy array, size n x p: parameter matrix
        y: 1D numpy array, size m: containing indices of correct outputs (zero-indexed)
        return_grad: boolean, whether or not to return gradients
    
    Returns: loss, or (loss, gradient) if return_grad = True
        loss: the average softmax loss over all examples
        gradient: gradient w.r.t. theta of average loss 
    """
    Y=np.array(y[:,None]==np.arange(Theta.shape[1])[None,:],dtype=np.float64)
    Yp=X.dot(Theta)
#     exp_Yp=np.exp(Yp)
    exp_Yp=np.exp(Yp-Yp.max(axis=1)[:,None]) # overflow
    sum_exp_Yp=np.sum(exp_Yp,axis=1)
    loss=(np.sum(np.log(sum_exp_Yp))-np.sum(Yp*Y))/X.shape[0] 
    if return_grad:
        G=X.T.dot(exp_Yp/sum_exp_Yp[:,None]-Y)/X.shape[0]
        return loss,G
    else:
        return loss

In [36]:
import time
m=X.shape[0]
split=int(math.floor(m*0.9))
P=np.random.permutation(m)
X_tr=X[P[0:split]]
X_te=X[P[split:m]]
y_tr=y[P[0:split]]
y_te=y[P[split:m]]
start=time.time()
loss, G = softmax_loss(X_tr,np.zeros((X_tr.shape[1], int(np.max(y_tr))+1)), y_tr, return_grad=True)
end=time.time()
print 'total time =',end-start
print loss, type(G), G.shape, np.linalg.norm(G)

total time = 0.018214225769
2.30258509299 <type 'numpy.ndarray'> (34, 10) 1.06245060377


In [37]:
def softmax_gd(X, y, X_test, y_test, lam=1e-5, alpha=1.0, iters=100):
    """
    Gradient descent to minimize softmax loss.
    
    Args:
        X : 2D numpy array, size m x n, array of all input features for all trainingexamples
        y: 1D numpy array, size m, contains indices of correct outputs on training set (zero-indexed)
        X_test : 2D numpy array, size m0 x n, array of all input features for all testing examples
        y_test: 1D numpy array, size m0, contains indices of correct outputs on testing set (zero-indexed)
        lam: regularization parameter
        alpha: step size
        iters: number of iterations of gradient descent
    
    Returns: Theta
        Theta: matrix of parameters for softmax model
    """
    Theta=np.zeros((X.shape[1],int(np.max(y))+1))
    for i in range(iters):
        loss,G=softmax_loss(X,Theta,y,return_grad=True)
        Theta-=alpha*(G+lam*Theta)
    return Theta      

In [38]:
start=time.time()
Theta = softmax_gd(X_tr, y_tr, X_te, y_te,iters=100)
print type(Theta), Theta.shape, np.linalg.norm(Theta)
end=time.time()
print 'time =',end-start

<type 'numpy.ndarray'> (34, 10) 48.3300809037
time = 1.54382610321


In [39]:
y_p=X_te.dot(Theta).argmax(axis=1)
accuracy=(y_p==y_te).sum()/float(len(y_te))
print 'accuracy =',accuracy

accuracy = 0.193151595745
